## Steps for generating object_detect.dlc

For this demo, a YoloNAS model is used. You can read more about this model in VisionSolution1-YoloNasSSD Readme.

#### Note->Use python3.8 or above for generating onnx and python3.6 for generating dlc

**Installing Necessary Libraries**

In [ ]:
!pip3 install super-gradients==3.1.2
!pip3 install cython

### Getting the dataset

In [ ]:
!wget https://github.com/ultralytics/yolov5/releases/download/v1.0/coco2017labels.zip -q --show-progress
!wget http://images.cocodataset.org/zips/val2017.zip -q --show-progress
!unzip val2017.zip
!unzip coco2017labels.zip


In [ ]:
files = os.listdir('val2017')
for file in files[50:]:
    os.remove("val2017/"+file)

In [ ]:
%%bash
rm -rf coco
rm -rf coco2017labels.zip
rm -rf val2017.zip

#### Downloading the YOLO_Nas Model

In [ ]:
## Downloading Model from git repo
import torch
# Load model with pretrained weights
from super_gradients.training import models
from super_gradients.common.object_names import Models

model = models.get(Models.YOLO_NAS_S, pretrained_weights="coco")

# Prepare model for conversion
# Input size is in format of [Batch x Channels x Width x Height] where 640 is the standard COCO dataset dimensions
model.eval()
model.prep_model_for_conversion(input_size=[1, 3, 320, 320])

# Create dummy_input
dummy_input = torch.randn([1, 3, 320, 320], device="cpu")

# Convert model to onnx
torch.onnx.export(model, dummy_input, "yolo_nas_s.onnx", opset_version=11)

#### Converting to DLC

In [32]:
import os
os.environ['SNPE_ROOT']="/local/mnt/workspace/snpe/snpe-2.20/2.20.0.240223/"

In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
snpe-onnx-to-dlc -i yolo_nas_s.onnx -o app/src/main/assets/yolo_nas_s.dlc

## Quantizing Yolo_nas

In [ ]:
##STEPS to preprocess images

def preprocess(original_image):
    resized_image = cv2.resize(original_image, (320, 320))
    resized_image = resized_image/255
    return resized_image

import cv2
import numpy as np
import os


dataset_path = "val2017/"

os.makedirs('rawYoloNAS', exist_ok=True)

filenames=[]
for path in os.listdir(dataset_path)[:5]:
    # check if current path is a file
    if os.path.isfile(os.path.join(dataset_path, path)):
        filenames.append(os.path.join(dataset_path, path))

for filename in filenames:
    original_image = cv2.imread(filename)
    img = preprocess(original_image)
    img = img.astype(np.float32)
    img.tofile("rawYoloNAS/"+filename.split("/")[-1].split(".")[0]+".raw")

In [ ]:
%%bash
find rawYoloNAS -name *.raw > YoloInputlist.txt
cat YoloInputlist.txt

In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
snpe-dlc-quantize --input_dlc app/src/main/assets/yolo_nas_s.dlc --input_list YoloInputlist.txt --use_enhanced_quantizer --use_adjusted_weights_quantizer --axis_quant --output_dlc app/src/main/assets/Quant_yoloNas_s_320.dlc

In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
snpe-dlc-graph-prepare --input_dlc app/src/main/assets/Quant_intermediate_yoloNas_s_320.dlc --set_output_tensors 885,877 --output_dlc app/src/main/assets/Quant_yoloNas_s_320.dlc --htp_socs sm8650


## How to change the object-detect model ? 

Object detection models are highly dependant on model architecture, and the pre-processing requirements vary a lot from model to model. 
If user intends to use a different model e.g. YoloV5, following steps should be followed : 

- Ensure Qualcomm® Neural Processing SDK supports the operations in selected model
- Study the pre processing, and post processing requirements for the selected model
- Most object detection models operate in RGB space. Input camera YUV buffers need to be converted to RGB basd on model requirements 


# Info about HRNET

HRNET model is State-of-the-art model for human pose estimation. It has good accuracy for results with single person, but has lower accuracy for multiple persons. To enhance that, HRNET uses object-detect model to identify a single person in a frame and then give the data to HRNET to get pose of that person. In this solution, we use MobileNetSSD for detecting human and then give the preprocesssed data to HRNET to achieve better accuracy for pose estimation.

HRNET dlc takes 256x192x3 flattened array as input and returns output of dims 17x64x48. HRNET generates heatmap for 17 human joints and each heatmap is of size 64x48.

In [43]:
%%bash
rm -rf HRNet-Human-Pose-Estimation/
git clone https://github.com/HRNet/HRNet-Human-Pose-Estimation.git
git checkout 00d7bf72f56382165e504b10ff0dddb82dca6fd2
cp hrnet.patch HRNet-Human-Pose-Estimation/
cd HRNet-Human-Pose-Estimation/
patch -p1 < ./hrnet.patch
cd lib
make

Cloning into 'HRNet-Human-Pose-Estimation'...
fatal: reference is not a tree: 00d7bf72f56382165e504b10ff0dddb82dca6fd2


patching file lib/nms/setup_linux.py
cd nms; python setup_linux.py build_ext --inplace; rm -rf build; cd ../../
running build_ext
Compiling cpu_nms.pyx because it changed.
[1/1] Cythonizing cpu_nms.pyx
building 'cpu_nms' extension
creating build
creating build/temp.linux-x86_64-cpython-38
x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -fPIC -I/local/mnt/workspace/sahinhos/sahin_env_3/lib/python3.8/site-packages/numpy/core/include -I/local/mnt/workspace/sahinhos/sahin_env_3/include -I/usr/include/python3.8 -c cpu_nms.c -o build/temp.linux-x86_64-cpython-38/cpu_nms.o -Wno-cpp -Wno-unused-function
x86_64-linux-gnu-gcc -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -g -fwrapv -O2 build/temp.linux-x86_64-cpython-38/cpu_nms.o -L/usr/lib/x86_64-linux-gnu -o /local/mnt/workspace/sahinhos/sahinworkspace/models-for-solutions/05-nlp-nlu/scripts/HRNet-Human-Pose-Esti

/local/mnt/workspace/sahinhos/sahin_env_3/lib/python3.8/site-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /local/mnt/workspace/sahinhos/sahinworkspace/models-for-solutions/05-nlp-nlu/scripts/HRNet-Human-Pose-Estimation/lib/nms/cpu_nms.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [45]:
import numpy as np
from matplotlib import pyplot as plt
import sys
import torch
import torch.utils.data
import torchvision.transforms as transforms
from config import cfg
import os
import os.path as osp
import urllib.request

%matplotlib inline


lib_path = osp.join(os.getcwd(), 'HRNet-Human-Pose-Estimation/lib')
sys.path.insert(0, lib_path)
if not os.path.exists("model_binaries"):
    os.makedirs("mode_binaries")
##Getting .pth file
OPTIMIZED_CHECKPOINT_URL = (
    "https://github.com/quic/aimet-model-zoo/releases/download/hrnet-posenet/"
)

if not os.path.exists(f"./model_binaries/hrnet_posenet_FP32.pth"):
    urllib.request.urlretrieve(
        f"{OPTIMIZED_CHECKPOINT_URL}/hrnet_posenet_FP32.pth",
        f"model_binaries/hrnet_posenet_FP32.pth",
    )


input_shape = (1, 3, 256, 192)
dummy_input = torch.randn(input_shape)
model = torch.load("model_binaries/hrnet_posenet_FP32.pth")
model.to('cpu')

onnx_model_name = "model_binaries/AIMET_HRNET_posnet.onnx"

opset = 11

torch.onnx.export(
    model.cpu(),
    dummy_input,
    onnx_model_name,
    verbose=True,
    do_constant_folding=True,
    export_params=True,
    input_names=['input'],
    output_names=['output'],
    opset_version=opset
)


graph(%input : Float(1, 3, 256, 192, strides=[147456, 49152, 192, 1], requires_grad=0, device=cpu),
      %final_layer.weight : Float(17, 32, 1, 1, strides=[32, 1, 1, 1], requires_grad=1, device=cpu),
      %final_layer.bias : Float(17, strides=[1], requires_grad=1, device=cpu),
      %2903 : Float(64, 3, 3, 3, strides=[27, 9, 3, 1], requires_grad=0, device=cpu),
      %2904 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %2906 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %2907 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %2909 : Float(64, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %2910 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %2912 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %2913 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %2915 : Float(256, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %2916 : F

## Steps for generating HRNET dlc for int8

In [46]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
snpe-onnx-to-dlc -i model_binaries/AIMET_HRNET_posnet.onnx -o app/src/main/assets/hrnet.dlc

[INFO] AISW SDK environment set
[INFO] SNPE_ROOT: /local/mnt/workspace/snpe/snpe-2.20/2.20.0.240223


2024-03-12 15:19:14,464 - 235 - INFO - Simplified model validation is successful
2024-03-12 15:19:17,882 - 235 - INFO - INFO_INITIALIZATION_SUCCESS: 
2024-03-12 15:19:18,265 - 235 - INFO - INFO_CONVERSION_SUCCESS: Conversion completed successfully
2024-03-12 15:19:18,531 - 235 - INFO - INFO_WRITE_SUCCESS: 


## Steps for Quantization

In [47]:
from PIL import Image
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
)

preproc = transforms.Compose(
        [
            transforms.ToTensor(),
            normalize,
        ]
    )

In [48]:

import cv2,os

dataset_path = "val2017/"

os.makedirs('rawHRNET', exist_ok=True)

filenames=[]
for path in os.listdir(dataset_path)[:5]:
    # check if current path is a file
    if os.path.isfile(os.path.join(dataset_path, path)):
        filenames.append(os.path.join(dataset_path, path))
print(filenames)

for filename in filenames:
    orig_img = cv2.imread(filename)
    img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(256,192),
                   interpolation = cv2.INTER_LINEAR)
    model_input = preproc(img).unsqueeze(0)

    model_input = model_input.cpu().detach().numpy()
    model_input = model_input.transpose(0,2,3,1)     
    fid = open("rawHRNET/"+filename.split("/")[-1].split(".")[0]+".raw", 'wb')
    model_input.tofile(fid)

['val2017/000000061108.jpg', 'val2017/000000388927.jpg', 'val2017/000000181499.jpg', 'val2017/000000458663.jpg', 'val2017/000000431545.jpg']


In [38]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh

find rawHRNET -name *.raw > HRNET_input_list.txt
snpe-dlc-quantize --input_dlc app/src/main/assets/hrnet.dlc --input_list HRNET_input_list.txt --axis_quant --output_dlc app/src/main/assets/hrnet_axis_int8.dlc --enable_htp --htp_socs sm8650
snpe-dlc-info --input_dlc app/src/main/assets/hrnet_axis_int8.dlc > hrnet_axis_int8.txt

[INFO] AISW SDK environment set
[INFO] SNPE_ROOT: /local/mnt/workspace/snpe/snpe-2.20/2.20.0.240223


[INFO] InitializeStderr: DebugLog initialized.
[WARNING] --axis_quant is deprecated, use --use_per_channel_quantization option.
[INFO] Processed command-line arguments
[INFO] Quantized parameters
[INFO] Generated activations
[INFO] Saved quantized dlc to: app/src/main/assets/hrnet_axis_int8.dlc
[INFO] DebugLog shutting down.


     0.2ms [  INFO ] Inferences will run in sync mode
     0.2ms [  INFO ] Initializing logging in the backend. Callback: [0xd15c30], Log Level: [3]
     0.2ms [  INFO ] No BackendExtensions lib provided;initializing NetRunBackend Interface
     0.3ms [WARNING] Unable to find a device with NetRunDeviceKeyDefault in Library NetRunBackendLibKeyDefault
     0.3ms [  INFO ] Entering QuantizeRuntimeApp flow
    62.0ms [  INFO ] CpuGraph::finalize
   118.9ms [  INFO ] CpuGraph::execute
   852.6ms [  INFO ] cleaning up resources for input tensors
   852.7ms [  INFO ] cleaning up resources for output tensors
   853.9ms [  INFO ] CpuGraph::execute
  1401.5ms [  INFO ] cleaning up resources for input tensors
  1401.5ms [  INFO ] cleaning up resources for output tensors
  1402.6ms [  INFO ] CpuGraph::execute
  2004.4ms [  INFO ] cleaning up resources for input tensors
  2004.4ms [  INFO ] cleaning up resources for output tensors
  2005.5ms [  INFO ] CpuGraph::execute
  2543.3ms [  INFO ] cleaning

[INFO] InitializeStderr: DebugLog initialized.
[INFO] SNPE HTP Offline Prepare: Attempting to create cache for SM8650
[USER_INFO] Target device backend record identifier: HTP_V75_SM8650_8MB
[USER_INFO] No cache record in the DLC matches the target device (HTP_V75_SM8650_8MB). Creating a new record
[INFO] Attempting to open dynamically linked lib: libHtpPrepare.so
[INFO] dlopen libHtpPrepare.so SUCCESS handle 0x1cfebd0
[INFO] Found Interface Provider (v2.14)
[USER_WARNING] QnnDsp <W> Initializing HtpProvider
[USER_WARNING] QnnDsp <W> HTP arch will be deprecated, please set SoC id instead.
[USER_WARNING] QnnDsp <W> Performance Estimates unsupported
[USER_INFO] Platform option not set
[USER_INFO] Created ctx=0x1 for Snpe Unique Graph ID=0 backend=3 instancePtr=0x1cee038
[USER_INFO] Offline Prepare VTCM size(MB) selected = 8
[USER_INFO] Offline Prepare Optimization Level passed = 2
[USER_INFO] Backend Mgr ~Dtor called for backend HTP
[USER_INFO] Cleaning up Context=0x1 for Snpe Unique Grap